# cache

Commands and population data for importing in to a database.

Even M$ visual studio products have to cache symbols somewhere.

I suppose this notebook should offline documentation too.

Rules:

- Cache functions and caches must be idempotent, calling the function multiple times with the same or different parameters will produce enjoyable results.
- Calling functions without any parameters should result in the default cache updating procedure, run for some time, and then exit.
- Caches stored in this notebook should be small, less than 1 MB each.
- Caches should be uniquely indexed and sorted so it is easy to display when something changes.



## notebook cache

Information about the current library cached.


### TODO: keyword index

A cache of question marked sentences to use as keywords in looking up functions.

TODO: look at media server caching techniques.


### TODO: imports cache

A cache of all imports in every cell, for making project trees.

TODO: use this cache to detect when a module changes in import notebook.


#### the cache

imports cache?

This code is automatically replaced with import code and stored in this notebook.


#### TODO: create import cache?



### exports cache

A cache of all exports, no matter the language.


#### the cache

exports cache?

This code is automatically replaced with export code and stored in this notebook.


In [ ]:

// export cache automatically replaced
var exportsCache = [
    [
        1563066646373,
        "2017 Update.ipynb[0]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066646374,
        "2017 Update.ipynb[1]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066646374,
        "2017 Update.ipynb[2]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066646374,
        "2017 Update.ipynb[3]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066646374,
        "2017 Update.ipynb[4]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066646414,
        "2017 Update.ipynb[5]",
        []
    ],
    [
        1563066617996,
        "avidbrain.ipynb[0]",
        [
            "testLogin"
        ]
    ],
    [
        1563066618062,
        "avidbrain.ipynb[1]",
        []
    ],
    [
        1563066618067,
        "avidbrain.ipynb[2]",
        []
    ],
    [
        1563066505731,
        "bash.ipynb[0]",
        "Line 1: Unexpected token |"
    ],
    [
        1563066505732,
        "bash.ipynb[1]",
        "Line 1: Unexpected token ~"
    ],
    [
        1563066505732,
        "bash.ipynb[2]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066505732,
        "bash.ipynb[3]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066505733,
        "bash.ipynb[4]",
        "Line 1: Unexpected token ~"
    ],
    [
        1563066505733,
        "bash.ipynb[5]",
        "Line 1: Unexpected token /"
    ],
    [
        1563066505733,
        "bash.ipynb[6]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066505733,
        "bash.ipynb[7]",
        "Line 1: Unexpected token :"
    ],
    [
        1563066643508,
        "brians resume.ipynb[0]",
        [
            "getBookmarkFolders"
        ]
    ],
    [
        1563066643862,
        "brians resume.ipynb[1]",
        [
            "updateInterestPage"
        ]
    ],
    [
        1563066503749,
        "child process.ipynb[0]",
        [
            "execCmd",
            "bashToExec"
        ]
    ],
    [
        1563066503778,
        "child process.ipynb[1]",
        []
    ],
    [
        1563066503844,
        "child process.ipynb[2]",
        [
            "bashToOne"
        ]
    ],
    [
        1563066495607,
        "compress.ipynb[0]",
        [
            "LZString",
            "o"
        ]
    ],
    [
        1563066501453,
        "compress.ipynb[1]",
        [
            "LZString",
            "getBaseValue"
        ]
    ],
    [
        1563066501470,
        "compress.ipynb[2]",
        []
    ],
    [
        1563066501473,
        "compress.ipynb[3]",
        []
    ],
    [
        1563066636093,
        "convert spreadsheet.ipynb[0]",
        []
    ],
    [
        1563066636622,
        "convert spreadsheet.ipynb[1]",
        [
            "filteredData",
            "unfilteredData"
        ]
    ],
    [
        1563066640036,
        "convert spreadsheet.ipynb[10]",
        [
            "getSheet",
            "safeName",
            "addSheet"
        ]
    ],
    [
        1563066640186,
        "convert spreadsheet.ipynb[11]",
        [
            "importSheet"
        ]
    ],
    [
        1563066640293,
        "convert spreadsheet.ipynb[12]",
        [
            "setupBackend"
        ]
    ],
    [
        1563066640396,
        "convert spreadsheet.ipynb[13]",
        [
            "copyMarketing"
        ]
    ],
    [
        1563066640397,
        "convert spreadsheet.ipynb[14]",
        "Line 2: Unexpected token :"
    ],
    [
        1563066637206,
        "convert spreadsheet.ipynb[2]",
        [
            "getTemplateProperties",
            "createAssignFilter"
        ]
    ],
    [
        1563066637665,
        "convert spreadsheet.ipynb[3]",
        [
            "renderRows",
            "safeName",
            "escape",
            "getDataClasses",
            "defineProperty"
        ]
    ],
    [
        1563066637952,
        "convert spreadsheet.ipynb[4]",
        [
            "wrapTemplate",
            "safeName",
            "toJSON"
        ]
    ],
    [
        1563066638336,
        "convert spreadsheet.ipynb[5]",
        [
            "collectRoutes"
        ]
    ],
    [
        1563066638933,
        "convert spreadsheet.ipynb[6]",
        [
            "collectExternalResources",
            "safeName"
        ]
    ],
    [
        1563066639039,
        "convert spreadsheet.ipynb[7]",
        [
            "getTemplateByUrl"
        ]
    ],
    [
        1563066639428,
        "convert spreadsheet.ipynb[8]",
        [
            "collectTemplateResources"
        ]
    ],
    [
        1563066639663,
        "convert spreadsheet.ipynb[9]",
        [
            "importTest"
        ]
    ],
    [
        1563066614353,
        "data collection.ipynb[0]",
        [
            "scrapeAlert"
        ]
    ],
    [
        1563066614850,
        "data collection.ipynb[1]",
        [
            "multiCrawl",
            "deQueue"
        ]
    ],
    [
        1563066615004,
        "data collection.ipynb[2]",
        [
            "searchResultsToJson"
        ]
    ],
    [
        1563066615184,
        "data collection.ipynb[3]",
        [
            "searchAll"
        ]
    ],
    [
        1563066615247,
        "data collection.ipynb[4]",
        [
            "scheduleSearch"
        ]
    ],
    [
        1563066615627,
        "data collection.ipynb[5]",
        [
            "crawlAll",
            "startCrawl",
            "fixLink"
        ]
    ],
    [
        1563066615700,
        "data collection.ipynb[6]",
        [
            "scheduleSearch"
        ]
    ],
    [
        1563066615842,
        "data collection.ipynb[7]",
        [
            "getJoke"
        ]
    ],
    [
        1563066486609,
        "dates.ipynb[0]",
        [
            "getTuesdays"
        ]
    ],
    [
        1563066486699,
        "dates.ipynb[1]",
        [
            "ISODateString",
            "pad"
        ]
    ],
    [
        1563066486827,
        "dates.ipynb[2]",
        [
            "sumEvents"
        ]
    ],
    [
        1563066486945,
        "dates.ipynb[3]",
        [
            "filterDistant"
        ]
    ],
    [
        1563066487233,
        "dates.ipynb[4]",
        [
            "graphDates"
        ]
    ],
    [
        1563066487237,
        "dates.ipynb[5]",
        []
    ],
    [
        1563066484060,
        "diff.ipynb[0]",
        [
            "diffTwoTexts",
            "htmlEntities",
            "prismHighlightHtml"
        ]
    ],
    [
        1563066484150,
        "diff.ipynb[1]",
        []
    ],
    [
        1563066484155,
        "diff.ipynb[2]",
        []
    ],
    [
        1563066633791,
        "ECommerce.ipynb[0]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066633792,
        "ECommerce.ipynb[1]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066633792,
        "ECommerce.ipynb[2]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066633792,
        "ECommerce.ipynb[3]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066633829,
        "ECommerce.ipynb[4]",
        []
    ],
    [
        1563066611346,
        "facebook connections.ipynb[0]",
        [
            "collectFacebookProfiles",
            "parseFacebookFriends"
        ]
    ],
    [
        1563066611399,
        "facebook connections.ipynb[1]",
        []
    ],
    [
        1563066611433,
        "facebook connections.ipynb[2]",
        []
    ],
    [
        1563066611554,
        "facebook connections.ipynb[3]",
        [
            "friendsOnFacebook"
        ]
    ],
    [
        1563066611995,
        "facebook connections.ipynb[4]",
        [
            "addFacebookFriends",
            "loadNewConnections"
        ]
    ],
    [
        1563066612013,
        "facebook connections.ipynb[5]",
        []
    ],
    [
        1563066606347,
        "facebook data.ipynb[0]",
        [
            "loginFacebook",
            "enterFacebook"
        ]
    ],
    [
        1563066606474,
        "facebook data.ipynb[1]",
        [
            "readFacebookProfileInfo"
        ]
    ],
    [
        1563066608451,
        "facebook data.ipynb[10]",
        []
    ],
    [
        1563066608561,
        "facebook data.ipynb[11]",
        [
            "clickInvite",
            "getUninvited",
            "clickConnect"
        ]
    ],
    [
        1563066607341,
        "facebook data.ipynb[2]",
        [
            "likeAllPosts",
            "listFacebookPosts",
            "scrapeFacebookPost",
            "likeFacebookPost"
        ]
    ],
    [
        1563066607665,
        "facebook data.ipynb[3]",
        [
            "scrapeFacebookEvent"
        ]
    ],
    [
        1563066607908,
        "facebook data.ipynb[4]",
        [
            "scrapeFacebookEvents",
            "getEvents"
        ]
    ],
    [
        1563066607923,
        "facebook data.ipynb[5]",
        []
    ],
    [
        1563066607970,
        "facebook data.ipynb[6]",
        [
            "scrapeFacebookFriends"
        ]
    ],
    [
        1563066608279,
        "facebook data.ipynb[7]",
        [
            "getFriendsDiff",
            "getFriends"
        ]
    ],
    [
        1563066608299,
        "facebook data.ipynb[8]",
        []
    ],
    [
        1563066608376,
        "facebook data.ipynb[9]",
        [
            "unfollowFacebook"
        ]
    ],
    [
        1563066602492,
        "facebook messaging.ipynb[0]",
        [
            "getUnreadThreads"
        ]
    ],
    [
        1563066602854,
        "facebook messaging.ipynb[1]",
        [
            "scanCommandsFacebook"
        ]
    ],
    [
        1563066602970,
        "facebook messaging.ipynb[2]",
        []
    ],
    [
        1563066603212,
        "facebook messaging.ipynb[3]",
        [
            "listFacebookThreads",
            "getFacebookThreads",
            "listArchivedFacebook"
        ]
    ],
    [
        1563066604032,
        "facebook messaging.ipynb[4]",
        [
            "readFacebookThread",
            "switchToParticipantThread",
            "getThreadParticipants",
            "readFacebookMessages"
        ]
    ],
    [
        1563066604059,
        "facebook messaging.ipynb[5]",
        []
    ],
    [
        1563066604178,
        "facebook messaging.ipynb[6]",
        [
            "sendFacebookMessage"
        ]
    ],
    [
        1563066599340,
        "forms.ipynb[0]",
        [
            "importer",
            "import"
        ]
    ],
    [
        1563066599456,
        "forms.ipynb[1]",
        [
            "selectDropdown"
        ]
    ],
    [
        1563066599553,
        "forms.ipynb[2]",
        [
            "fillForm"
        ]
    ],
    [
        1563066599991,
        "forms.ipynb[3]",
        [
            "multiLogin",
            "fillAll"
        ]
    ],
    [
        1563066600193,
        "forms.ipynb[4]",
        [
            "testLogins"
        ]
    ],
    [
        1563066600200,
        "forms.ipynb[5]",
        []
    ],
    [
        1563066597254,
        "github.ipynb[0]",
        [
            "getGithub"
        ]
    ],
    [
        1563066597260,
        "github.ipynb[1]",
        []
    ],
    [
        1563066597264,
        "github.ipynb[2]",
        []
    ],
    [
        1563066631590,
        "Good UX Intro.ipynb[0]",
        []
    ],
    [
        1563066631631,
        "Good UX Intro.ipynb[1]",
        []
    ],
    [
        1563066631672,
        "Good UX Intro.ipynb[2]",
        []
    ],
    [
        1563066631707,
        "Good UX Intro.ipynb[3]",
        []
    ],
    [
        1563066631715,
        "Good UX Intro.ipynb[4]",
        []
    ],
    [
        1563066594744,
        "google authorize.ipynb[0]",
        [
            "authorize",
            "storeToken",
            "receiveCode",
            "errorFallback",
            "renewToken"
        ]
    ],
    [
        1563066594846,
        "google authorize.ipynb[1]",
        [
            "authorizeSelenium"
        ]
    ],
    [
        1563066594963,
        "google authorize.ipynb[2]",
        []
    ],
    [
        1563066594967,
        "google authorize.ipynb[3]",
        []
    ],
    [
        1563066674640,
        "google calendar data.ipynb[0]",
        []
    ],
    [
        1563066674687,
        "google calendar data.ipynb[1]",
        [
            "chromeDtToDate"
        ]
    ],
    [
        1563066674915,
        "google calendar data.ipynb[2]",
        [
            "getBookmarksFromTakeout"
        ]
    ],
    [
        1563066675018,
        "google calendar data.ipynb[3]",
        [
            "getHistory"
        ]
    ],
    [
        1563066675050,
        "google calendar data.ipynb[4]",
        []
    ],
    [
        1563066675274,
        "google calendar data.ipynb[5]",
        [
            "syncChromeBookmarks"
        ]
    ],
    [
        1563066675814,
        "google calendar data.ipynb[6]",
        [
            "syncChromeHistory",
            "loadChromeHistory"
        ]
    ],
    [
        1563066676059,
        "google calendar data.ipynb[7]",
        [
            "syncChrome",
            "unzip"
        ]
    ],
    [
        1563066676086,
        "google calendar data.ipynb[8]",
        []
    ],
    [
        1563066671641,
        "google calendar graphs.ipynb[0]",
        []
    ],
    [
        1563066671855,
        "google calendar graphs.ipynb[1]",
        [
            "calendarSearchToHeatmap"
        ]
    ],
    [
        1563066672114,
        "google calendar graphs.ipynb[2]",
        [
            "calendarSearchToSwimlane"
        ]
    ],
    [
        1563066672264,
        "google calendar graphs.ipynb[3]",
        [
            "calendarSearchToPieChart"
        ]
    ],
    [
        1563066672268,
        "google calendar graphs.ipynb[4]",
        []
    ],
    [
        1563066667447,
        "google calendar.ipynb[0]",
        [
            "authorizeCalendar"
        ]
    ],
    [
        1563066667765,
        "google calendar.ipynb[1]",
        [
            "listEvents",
            "processResult"
        ]
    ],
    [
        1563066668057,
        "google calendar.ipynb[2]",
        [
            "runTodaysEvents"
        ]
    ],
    [
        1563066668119,
        "google calendar.ipynb[3]",
        []
    ],
    [
        1563066668168,
        "google calendar.ipynb[4]",
        [
            "correctTimeLimits"
        ]
    ],
    [
        1563066668426,
        "google calendar.ipynb[5]",
        [
            "correctCalendarId",
            "filterCalendar"
        ]
    ],
    [
        1563066668688,
        "google calendar.ipynb[6]",
        [
            "getDaysEvents"
        ]
    ],
    [
        1563066669183,
        "google calendar.ipynb[7]",
        [
            "updateEvent"
        ]
    ],
    [
        1563066669293,
        "google calendar.ipynb[8]",
        [
            "createNewEvent"
        ]
    ],
    [
        1563066664818,
        "google contacts.ipynb[0]",
        [
            "getContacts",
            "findMatch",
            "googlePromise",
            "listContacts"
        ]
    ],
    [
        1563066665369,
        "google contacts.ipynb[1]",
        [
            "getSettings",
            "googlePromise",
            "mapMembership"
        ]
    ],
    [
        1563066665395,
        "google contacts.ipynb[2]",
        []
    ],
    [
        1563066665399,
        "google contacts.ipynb[3]",
        []
    ],
    [
        1563066662087,
        "google drive.ipynb[0]",
        [
            "authorizeDrive"
        ]
    ],
    [
        1563066662157,
        "google drive.ipynb[1]",
        []
    ],
    [
        1563066662210,
        "google drive.ipynb[2]",
        [
            "listDrive"
        ]
    ],
    [
        1563066662280,
        "google drive.ipynb[3]",
        [
            "insertPermission"
        ]
    ],
    [
        1563066662304,
        "google drive.ipynb[4]",
        []
    ],
    [
        1563066662389,
        "google drive.ipynb[5]",
        [
            "createSheet"
        ]
    ],
    [
        1563066662444,
        "google drive.ipynb[6]",
        [
            "copyFile"
        ]
    ],
    [
        1563066659566,
        "google maps.ipynb[0]",
        [
            "placesNearby"
        ]
    ],
    [
        1563066659615,
        "google maps.ipynb[1]",
        [
            "placeDetails"
        ]
    ],
    [
        1563066659677,
        "google maps.ipynb[2]",
        [
            "googleGeocodeAddress"
        ]
    ],
    [
        1563066659922,
        "google maps.ipynb[3]",
        [
            "getNearby"
        ]
    ],
    [
        1563066659926,
        "google maps.ipynb[4]",
        []
    ],
    [
        1563066656366,
        "google scheduling.ipynb[0]",
        [
            "adjustPurpleEvents",
            "googlePromise",
            "batchPromises"
        ]
    ],
    [
        1563066656373,
        "google scheduling.ipynb[1]",
        []
    ],
    [
        1563066656378,
        "google scheduling.ipynb[2]",
        []
    ],
    [
        1563066656383,
        "google scheduling.ipynb[3]",
        []
    ],
    [
        1563066656391,
        "google scheduling.ipynb[4]",
        []
    ],
    [
        1563066656941,
        "google scheduling.ipynb[5]",
        [
            "calendarHeatmap"
        ]
    ],
    [
        1563066657087,
        "google scheduling.ipynb[6]",
        [
            "searchCalendar"
        ]
    ],
    [
        1563066657092,
        "google scheduling.ipynb[7]",
        []
    ],
    [
        1563066652519,
        "google search api.ipynb[0]",
        [
            "search"
        ]
    ],
    [
        1563066652586,
        "google search api.ipynb[1]",
        []
    ],
    [
        1563066652721,
        "google search api.ipynb[2]",
        [
            "authorizeSearch"
        ]
    ],
    [
        1563066648892,
        "google sheets.ipynb[0]",
        [
            "getTemplates"
        ]
    ],
    [
        1563066649048,
        "google sheets.ipynb[1]",
        []
    ],
    [
        1563066650150,
        "google sheets.ipynb[10]",
        [
            "authorizeSheets"
        ]
    ],
    [
        1563066649141,
        "google sheets.ipynb[2]",
        [
            "getInfo"
        ]
    ],
    [
        1563066649215,
        "google sheets.ipynb[3]",
        [
            "getDataSheet"
        ]
    ],
    [
        1563066649366,
        "google sheets.ipynb[4]",
        [
            "getRows"
        ]
    ],
    [
        1563066649487,
        "google sheets.ipynb[5]",
        []
    ],
    [
        1563066649546,
        "google sheets.ipynb[6]",
        [
            "addRow"
        ]
    ],
    [
        1563066649656,
        "google sheets.ipynb[7]",
        [
            "getSheetByName"
        ]
    ],
    [
        1563066650004,
        "google sheets.ipynb[8]",
        [
            "updateRow"
        ]
    ],
    [
        1563066650061,
        "google sheets.ipynb[9]",
        []
    ],
    [
        1563066590951,
        "google takeout.ipynb[0]",
        [
            "downloadGoogleTakeout",
            "listTakeouts",
            "takeoutProducts"
        ]
    ],
    [
        1563066590992,
        "google takeout.ipynb[1]",
        [
            "googleTakeout"
        ]
    ],
    [
        1563066591027,
        "google takeout.ipynb[2]",
        []
    ],
    [
        1563066591558,
        "google takeout.ipynb[3]",
        [
            "convertUnicode",
            "addSite"
        ]
    ],
    [
        1563066591561,
        "google takeout.ipynb[4]",
        []
    ],
    [
        1563066584711,
        "google timeline.ipynb[0]",
        [
            "getGoogleTimeline",
            "selectDate"
        ]
    ],
    [
        1563066585376,
        "google timeline.ipynb[1]",
        [
            "readTimelinePage"
        ]
    ],
    [
        1563066585923,
        "google timeline.ipynb[2]",
        [
            "averageDestinations",
            "toRadians",
            "straightDistance"
        ]
    ],
    [
        1563066586815,
        "google timeline.ipynb[3]",
        [
            "reconcileTimelineLocations",
            "filterEvents"
        ]
    ],
    [
        1563066587033,
        "google timeline.ipynb[4]",
        [
            "loadLocations"
        ]
    ],
    [
        1563066587768,
        "google timeline.ipynb[5]",
        [
            "reconcileTimeline",
            "loadOnce",
            "daysInMonth"
        ]
    ],
    [
        1563066587897,
        "google timeline.ipynb[6]",
        [
            "timelineEvents"
        ]
    ],
    [
        1563066481222,
        "heartbeat.ipynb[0]",
        [
            "thump"
        ]
    ],
    [
        1563066481520,
        "heartbeat.ipynb[1]",
        [
            "runTodaysHeartbeat"
        ]
    ],
    [
        1563066481525,
        "heartbeat.ipynb[2]",
        []
    ],
    [
        1563066628851,
        "Landing Pages.ipynb[0]",
        []
    ],
    [
        1563066628884,
        "Landing Pages.ipynb[1]",
        []
    ],
    [
        1563066629095,
        "Landing Pages.ipynb[10]",
        []
    ],
    [
        1563066629113,
        "Landing Pages.ipynb[11]",
        []
    ],
    [
        1563066629130,
        "Landing Pages.ipynb[12]",
        []
    ],
    [
        1563066629149,
        "Landing Pages.ipynb[13]",
        []
    ],
    [
        1563066629167,
        "Landing Pages.ipynb[14]",
        []
    ],
    [
        1563066629180,
        "Landing Pages.ipynb[15]",
        []
    ],
    [
        1563066629199,
        "Landing Pages.ipynb[16]",
        []
    ],
    [
        1563066629216,
        "Landing Pages.ipynb[17]",
        []
    ],
    [
        1563066629230,
        "Landing Pages.ipynb[18]",
        []
    ],
    [
        1563066629245,
        "Landing Pages.ipynb[19]",
        []
    ],
    [
        1563066628913,
        "Landing Pages.ipynb[2]",
        []
    ],
    [
        1563066629249,
        "Landing Pages.ipynb[20]",
        []
    ],
    [
        1563066628939,
        "Landing Pages.ipynb[3]",
        []
    ],
    [
        1563066628959,
        "Landing Pages.ipynb[4]",
        []
    ],
    [
        1563066628983,
        "Landing Pages.ipynb[5]",
        []
    ],
    [
        1563066629011,
        "Landing Pages.ipynb[6]",
        []
    ],
    [
        1563066629033,
        "Landing Pages.ipynb[7]",
        []
    ],
    [
        1563066629053,
        "Landing Pages.ipynb[8]",
        []
    ],
    [
        1563066629075,
        "Landing Pages.ipynb[9]",
        []
    ],
    [
        1563066478914,
        "levenshtein.ipynb[0]",
        [
            "levDist"
        ]
    ],
    [
        1563066478980,
        "levenshtein.ipynb[1]",
        [
            "levSort"
        ]
    ],
    [
        1563066478984,
        "levenshtein.ipynb[2]",
        []
    ],
    [
        1563066580442,
        "linkedin connections.ipynb[0]",
        [
            "syncLinkedInContacts",
            "escapeFilename"
        ]
    ],
    [
        1563066581014,
        "linkedin connections.ipynb[1]",
        [
            "scrapeEntireLinkedInProfile",
            "loadEntirePage"
        ]
    ],
    [
        1563066581189,
        "linkedin connections.ipynb[2]",
        [
            "listAllConnections"
        ]
    ],
    [
        1563066581223,
        "linkedin connections.ipynb[3]",
        []
    ],
    [
        1563066581598,
        "linkedin connections.ipynb[4]",
        [
            "listAllConnections",
            "connectLinkedin"
        ]
    ],
    [
        1563066581819,
        "linkedin connections.ipynb[5]",
        [
            "addLinkedinConnections"
        ]
    ],
    [
        1563066581855,
        "linkedin connections.ipynb[6]",
        []
    ],
    [
        1563066581859,
        "linkedin connections.ipynb[7]",
        []
    ],
    [
        1563066575852,
        "linkedin messages.ipynb[0]",
        [
            "loginLinkedIn",
            "enterLinkedIn"
        ]
    ],
    [
        1563066576117,
        "linkedin messages.ipynb[1]",
        [
            "readLinkedInProfileInfo",
            "visitMyProfile"
        ]
    ],
    [
        1563066576268,
        "linkedin messages.ipynb[2]",
        [
            "listLinkedInThreads",
            "scrollLinkedInThreads",
            "getLinkedInThreads"
        ]
    ],
    [
        1563066577003,
        "linkedin messages.ipynb[3]",
        [
            "readLinkedInThread",
            "getThreadParticipants",
            "scrollLinkedInMessages",
            "readLinkedInMessages"
        ]
    ],
    [
        1563066577474,
        "linkedin messages.ipynb[4]",
        [
            "scrapeLinkedInThreads",
            "readThread",
            "listThreads"
        ]
    ],
    [
        1563066577496,
        "linkedin messages.ipynb[5]",
        []
    ],
    [
        1563066577500,
        "linkedin messages.ipynb[6]",
        []
    ],
    [
        1563066626467,
        "marketing scripts.ipynb[0]",
        [
            "checkout"
        ]
    ],
    [
        1563066626601,
        "marketing scripts.ipynb[1]",
        [
            "contactUs"
        ]
    ],
    [
        1563066626602,
        "marketing scripts.ipynb[2]",
        "Line 2: Unexpected token :"
    ],
    [
        1563066476271,
        "mono.ipynb[0]",
        []
    ],
    [
        1563066476418,
        "mono.ipynb[1]",
        []
    ],
    [
        1563066476419,
        "mono.ipynb[2]",
        "Line 1: Unexpected token ILLEGAL"
    ],
    [
        1563066473940,
        "nmap.ipynb[0]",
        []
    ],
    [
        1563066473946,
        "nmap.ipynb[1]",
        []
    ],
    [
        1563066702355,
        "notebook extensions.ipynb[0]",
        []
    ],
    [
        1563066471875,
        "openssl.ipynb[0]",
        "Line 3: Unexpected token ILLEGAL"
    ],
    [
        1563066471876,
        "openssl.ipynb[1]",
        "Line 3: Unexpected token ILLEGAL"
    ],
    [
        1563066471876,
        "openssl.ipynb[2]",
        "Line 3: Unexpected token ILLEGAL"
    ],
    [
        1563066471912,
        "openssl.ipynb[3]",
        []
    ],
    [
        1563066571372,
        "orchestration.ipynb[0]",
        [
            "resizeWindow",
            "getScreenSize"
        ]
    ],
    [
        1563066571650,
        "orchestration.ipynb[1]",
        [
            "onlyOneWindow",
            "closeAllTabs",
            "closeAllWindows"
        ]
    ],
    [
        1563066571927,
        "orchestration.ipynb[2]",
        [
            "tileWindows",
            "openUrl",
            "createNewWindows"
        ]
    ],
    [
        1563066572124,
        "orchestration.ipynb[3]",
        [
            "getAllSessionUrls",
            "repositionSession"
        ]
    ],
    [
        1563066572159,
        "orchestration.ipynb[4]",
        [
            "sendJoke"
        ]
    ],
    [
        1563066572217,
        "orchestration.ipynb[5]",
        [
            "sendFacebookThanks"
        ]
    ],
    [
        1563066572442,
        "orchestration.ipynb[6]",
        [
            "tellJokes"
        ]
    ],
    [
        1563066572664,
        "orchestration.ipynb[7]",
        [
            "cleanUpSessions",
            "closeAllSessions"
        ]
    ],
    [
        1563066573224,
        "orchestration.ipynb[8]",
        [
            "getScreenshots",
            "uploadS3"
        ]
    ],
    [
        1563066573227,
        "orchestration.ipynb[9]",
        []
    ],
    [
        1563066469861,
        "parse.ipynb[0]",
        [
            "escapeRegExp"
        ]
    ],
    [
        1563066467048,
        "polyfills.ipynb[0]",
        [
            "request"
        ]
    ],
    [
        1563066467056,
        "polyfills.ipynb[1]",
        []
    ],
    [
        1563066699271,
        "promisify automock socketio.ipynb[0]",
        [
            "promisifyMock"
        ]
    ],
    [
        1563066700107,
        "promisify automock socketio.ipynb[1]",
        [
            "sockifyRequire",
            "sockifyServer"
        ]
    ],
    [
        1563066700218,
        "promisify automock socketio.ipynb[2]",
        [
            "sockifyClient"
        ]
    ],
    [
        1563066700256,
        "promisify automock socketio.ipynb[3]",
        []
    ],
    [
        1563066700315,
        "promisify automock socketio.ipynb[4]",
        []
    ],
    [
        1563066700320,
        "promisify automock socketio.ipynb[5]",
        []
    ],
    [
        1563066464153,
        "qr codes.ipynb[0]",
        []
    ],
    [
        1563066464440,
        "qr codes.ipynb[1]",
        [
            "httpRequest"
        ]
    ],
    [
        1563066464447,
        "qr codes.ipynb[2]",
        []
    ],
    [
        1563066697027,
        "register jupter kernels.ipynb[0]",
        "Line 1: Unexpected token"
    ],
    [
        1563066460610,
        "scraping.ipynb[0]",
        [
            "getLocations",
            "getLocation"
        ]
    ],
    [
        1563066460954,
        "scraping.ipynb[1]",
        [
            "getNearbyJSON",
            "getResultsPage",
            "getAllResults"
        ]
    ],
    [
        1563066461892,
        "scraping.ipynb[2]",
        [
            "loadLocations",
            "getAllLocationsData"
        ]
    ],
    [
        1563066461895,
        "scraping.ipynb[3]",
        []
    ],
    [
        1563066569097,
        "selenium commands.ipynb[0]",
        "Line 1: Unexpected number"
    ],
    [
        1563066569098,
        "selenium commands.ipynb[1]",
        "Line 1: Unexpected token"
    ],
    [
        1563066569098,
        "selenium commands.ipynb[2]",
        "Line 1: Unexpected token"
    ],
    [
        1563066569098,
        "selenium commands.ipynb[3]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066567168,
        "selenium demo.ipynb[0]",
        []
    ],
    [
        1563066567178,
        "selenium demo.ipynb[1]",
        []
    ],
    [
        1563066527255,
        "selenium server.ipynb[0]",
        [
            "seleniumServer",
            "promisifyChrome",
            "response"
        ]
    ],
    [
        1563066564161,
        "selenium server.ipynb[1]",
        [
            "router",
            "route"
        ]
    ],
    [
        1563066564458,
        "selenium server.ipynb[2]",
        [
            "go",
            "router",
            "waitForComplete"
        ]
    ],
    [
        1563066564568,
        "selenium server.ipynb[3]",
        [
            "findElements",
            "router"
        ]
    ],
    [
        1563066564767,
        "selenium server.ipynb[4]",
        [
            "deleteSession",
            "createSession",
            "router",
            "getSessionCapabilities"
        ]
    ],
    [
        1563066564917,
        "selenium server.ipynb[5]",
        [
            "getWindowHandles",
            "router"
        ]
    ],
    [
        1563066565050,
        "selenium server.ipynb[6]",
        [
            "clickElement",
            "router"
        ]
    ],
    [
        1563066524889,
        "send social message.ipynb[0]",
        []
    ],
    [
        1563066521853,
        "service auth.ipynb[0]",
        [
            "getCredentials",
            "decrypt"
        ]
    ],
    [
        1563066522199,
        "service auth.ipynb[1]",
        [
            "saveCredentials",
            "encrypt",
            "encryptSet"
        ]
    ],
    [
        1563066522220,
        "service auth.ipynb[2]",
        []
    ],
    [
        1563066522414,
        "service auth.ipynb[3]",
        [
            "readPasswordsHtm"
        ]
    ],
    [
        1563066522612,
        "service auth.ipynb[4]",
        [
            "downloadGooglePasswords",
            "waitForPasswordLoad",
            "copyPasswordRow",
            "copyPasswords"
        ]
    ],
    [
        1563066522637,
        "service auth.ipynb[5]",
        []
    ],
    [
        1563066522979,
        "service auth.ipynb[6]",
        [
            "loginGoogle",
            "enterGoogleUsername",
            "enterGooglePassword",
            "enterCredentials"
        ]
    ],
    [
        1563066522982,
        "service auth.ipynb[7]",
        []
    ],
    [
        1563066622848,
        "study sauce.ipynb[0]",
        []
    ],
    [
        1563066623036,
        "study sauce.ipynb[1]",
        [
            "getSignedUrl",
            "req",
            "res",
            "method",
            "status",
            "end",
            "file",
            "storage",
            "bucket",
            "body",
            "filename",
            "expiresAtMs",
            "Date",
            "now",
            "config",
            "action",
            "expires",
            "contentType",
            "err",
            "url",
            "console",
            "error",
            "send"
        ]
    ],
    [
        1563066623160,
        "study sauce.ipynb[2]",
        [
            "copyStudy"
        ]
    ],
    [
        1563066623514,
        "study sauce.ipynb[3]",
        [
            "renderCards",
            "parseCards"
        ]
    ],
    [
        1563066623773,
        "study sauce.ipynb[4]",
        [
            "authorizeProfile"
        ]
    ],
    [
        1563066623992,
        "study sauce.ipynb[5]",
        [
            "receiveCode",
            "extractProfile",
            "safeName"
        ]
    ],
    [
        1563066624155,
        "study sauce.ipynb[6]",
        [
            "renderUser"
        ]
    ],
    [
        1563066624155,
        "study sauce.ipynb[7]",
        "Line 2: Unexpected token :"
    ],
    [
        1563066624189,
        "study sauce.ipynb[8]",
        [
            "createStudyPack",
            "createCards"
        ]
    ],
    [
        1563066694892,
        "test runner.ipynb[0]",
        [
            "testCells",
            "resetTests"
        ]
    ],
    [
        1563066695013,
        "test runner.ipynb[1]",
        [
            "testWatcher"
        ]
    ],
    [
        1563066692035,
        "twilio.ipynb[0]",
        [
            "incomingTwilio"
        ]
    ],
    [
        1563066692277,
        "twilio.ipynb[1]",
        [
            "incomingTwilio",
            "setReminder"
        ]
    ],
    [
        1563066692375,
        "twilio.ipynb[2]",
        [
            "sendTwilio"
        ]
    ],
    [
        1563066692452,
        "twilio.ipynb[3]",
        [
            "callTwilio"
        ]
    ],
    [
        1563066507643,
        "Untitled.ipynb[0]",
        []
    ],
    [
        1563066507648,
        "Untitled.ipynb[1]",
        []
    ],
    [
        1563066519119,
        "utilities.ipynb[0]",
        [
            "scrollClient"
        ]
    ],
    [
        1563066519340,
        "utilities.ipynb[1]",
        [
            "getAllUntil"
        ]
    ],
    [
        1563066519495,
        "utilities.ipynb[2]",
        [
            "clickSpa"
        ]
    ],
    [
        1563066519500,
        "utilities.ipynb[3]",
        []
    ],
    [
        1563066458154,
        "uuid.ipynb[0]",
        [
            "v35",
            "sha1",
            "bytesToUuid",
            "uuidToBytes",
            "stringToBytes"
        ]
    ],
    [
        1563066455504,
        "vnc.ipynb[0]",
        "Line 1: Unexpected token :"
    ],
    [
        1563066455504,
        "vnc.ipynb[1]",
        "Line 1: Unexpected token ."
    ],
    [
        1563066455504,
        "vnc.ipynb[2]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066455505,
        "vnc.ipynb[3]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066455505,
        "vnc.ipynb[4]",
        "Line 2: Unexpected identifier"
    ],
    [
        1563066455505,
        "vnc.ipynb[5]",
        "Line 1: Unexpected string"
    ],
    [
        1563066455505,
        "vnc.ipynb[6]",
        "Line 1: Unexpected token ILLEGAL"
    ],
    [
        1563066455506,
        "vnc.ipynb[7]",
        "Line 1: Unexpected token ~"
    ],
    [
        1563066455506,
        "vnc.ipynb[8]",
        "Line 1: Unexpected token ~"
    ],
    [
        1563066515459,
        "webdriver.ipynb[0]",
        [
            "createWebdriverClient"
        ]
    ],
    [
        1563066515746,
        "webdriver.ipynb[1]",
        [
            "connectSession"
        ]
    ],
    [
        1563066515865,
        "webdriver.ipynb[2]",
        [
            "readSessions"
        ]
    ],
    [
        1563066516210,
        "webdriver.ipynb[3]",
        [
            "updateOrAddSession",
            "lockPromise"
        ]
    ],
    [
        1563066516433,
        "webdriver.ipynb[4]",
        [
            "getSessions",
            "lockPromise",
            "updateOrAddSession"
        ]
    ],
    [
        1563066516694,
        "webdriver.ipynb[5]",
        [
            "lockPromise",
            "verifySession",
            "updateOrAddSession",
            "scanning",
            "addPlugins"
        ]
    ],
    [
        1563066516715,
        "webdriver.ipynb[6]",
        []
    ],
    [
        1563066516739,
        "webdriver.ipynb[7]",
        []
    ],
    [
        1563066516747,
        "webdriver.ipynb[8]",
        []
    ],
    [
        1563066516750,
        "webdriver.ipynb[9]",
        []
    ],
    [
        1563066512574,
        "What is Selenium.ipynb[0]",
        [
            "getSeleniumServer"
        ]
    ],
    [
        1563066512742,
        "What is Selenium.ipynb[1]",
        [
            "vncIframe",
            "urlEncode"
        ]
    ],
    [
        1563066512902,
        "What is Selenium.ipynb[2]",
        [
            "runSeleniumCell"
        ]
    ],
    [
        1563066512921,
        "What is Selenium.ipynb[3]",
        [
            "testLive"
        ]
    ],
    [
        1563066512926,
        "What is Selenium.ipynb[4]",
        []
    ],
    [
        1563066453567,
        "windows.ipynb[0]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066453568,
        "windows.ipynb[1]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066453602,
        "windows.ipynb[2]",
        []
    ],
    [
        1563066453602,
        "windows.ipynb[3]",
        "Line 1: Unexpected token <"
    ],
    [
        1563066453607,
        "windows.ipynb[4]",
        []
    ],
    [
        1563066453607,
        "windows.ipynb[5]",
        "Line 2: Unexpected identifier"
    ],
    [
        1563066453611,
        "windows.ipynb[6]",
        []
    ],
    [
        1563066453612,
        "windows.ipynb[7]",
        "Line 1: Unexpected string"
    ],
    [
        1563066620094,
        "Wireframing.ipynb[0]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066620154,
        "Wireframing.ipynb[1]",
        []
    ],
    [
        1563066620694,
        "Wireframing.ipynb[10]",
        []
    ],
    [
        1563066620843,
        "Wireframing.ipynb[11]",
        []
    ],
    [
        1563066620843,
        "Wireframing.ipynb[12]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066620843,
        "Wireframing.ipynb[13]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066620846,
        "Wireframing.ipynb[14]",
        []
    ],
    [
        1563066620167,
        "Wireframing.ipynb[2]",
        []
    ],
    [
        1563066620235,
        "Wireframing.ipynb[3]",
        [
            "npmls"
        ]
    ],
    [
        1563066620277,
        "Wireframing.ipynb[4]",
        []
    ],
    [
        1563066620361,
        "Wireframing.ipynb[5]",
        []
    ],
    [
        1563066620396,
        "Wireframing.ipynb[6]",
        []
    ],
    [
        1563066620403,
        "Wireframing.ipynb[7]",
        []
    ],
    [
        1563066620448,
        "Wireframing.ipynb[8]",
        []
    ],
    [
        1563066620542,
        "Wireframing.ipynb[9]",
        []
    ],
    [
        1563066509961,
        "you earned it.ipynb[0]",
        [
            "highFive",
            "loginYouEarnedIt",
            "doHighFive"
        ]
    ],
    [
        1563066509997,
        "you earned it.ipynb[1]",
        [
            "runHighFiver"
        ]
    ],
    [
        1563066510018,
        "you earned it.ipynb[2]",
        []
    ],
    [
        1563066510023,
        "you earned it.ipynb[3]",
        []
    ],
    [
        1563066678709,
        "zuora to eloqua.ipynb[0]",
        [
            "csvToJson",
            "createBulkExportJob",
            "getBulkExportFile",
            "getBulkExportJobStatus",
            "getAuthHeaders"
        ]
    ],
    [
        1563066678803,
        "zuora to eloqua.ipynb[1]",
        [
            "getCatalog"
        ]
    ],
    [
        1563066682511,
        "zuora to eloqua.ipynb[10]",
        [
            "bulkImportTemplate",
            "contentCreateTemplate",
            "temporaryImportTemplate"
        ]
    ],
    [
        1563066682553,
        "zuora to eloqua.ipynb[11]",
        []
    ],
    [
        1563066682667,
        "zuora to eloqua.ipynb[12]",
        [
            "getImportData",
            "getOauthToken",
            "getEloquaConfig"
        ]
    ],
    [
        1563066682812,
        "zuora to eloqua.ipynb[13]",
        [
            "getCustomDataObject",
            "getImportDefinitions"
        ]
    ],
    [
        1563066683031,
        "zuora to eloqua.ipynb[14]",
        []
    ],
    [
        1563066683190,
        "zuora to eloqua.ipynb[15]",
        [
            "handler"
        ]
    ],
    [
        1563066683468,
        "zuora to eloqua.ipynb[16]",
        []
    ],
    [
        1563066683685,
        "zuora to eloqua.ipynb[17]",
        [
            "handler"
        ]
    ],
    [
        1563066683950,
        "zuora to eloqua.ipynb[18]",
        []
    ],
    [
        1563066684156,
        "zuora to eloqua.ipynb[19]",
        [
            "getZuoraMonth"
        ]
    ],
    [
        1563066679111,
        "zuora to eloqua.ipynb[2]",
        []
    ],
    [
        1563066684619,
        "zuora to eloqua.ipynb[20]",
        []
    ],
    [
        1563066684833,
        "zuora to eloqua.ipynb[21]",
        [
            "getZuoraAccounts",
            "zuoraQuery",
            "getContact",
            "getAccountById",
            "getPaymentMethod",
            "getAccountLast4Digits"
        ]
    ],
    [
        1563066684942,
        "zuora to eloqua.ipynb[22]",
        []
    ],
    [
        1563066685130,
        "zuora to eloqua.ipynb[23]",
        [
            "bulkUploadEloqua"
        ]
    ],
    [
        1563066685917,
        "zuora to eloqua.ipynb[24]",
        []
    ],
    [
        1563066685941,
        "zuora to eloqua.ipynb[25]",
        []
    ],
    [
        1563066686274,
        "zuora to eloqua.ipynb[26]",
        [
            "calculatePrice"
        ]
    ],
    [
        1563066688867,
        "zuora to eloqua.ipynb[27]",
        [
            "filterROR",
            "rorsToAccounts",
            "totalFilteredRecords",
            "accountTotals",
            "verifyMissing",
            "validateWorksheet",
            "compareRecordsCatalog"
        ]
    ],
    [
        1563066688870,
        "zuora to eloqua.ipynb[28]",
        []
    ],
    [
        1563066689289,
        "zuora to eloqua.ipynb[29]",
        [
            "zuoraQuery",
            "getContact",
            "getContactByAccount",
            "getAccountById",
            "getAccount",
            "getSubscription",
            "getPaymentMethod",
            "getRatePlans"
        ]
    ],
    [
        1563066679240,
        "zuora to eloqua.ipynb[3]",
        [
            "getQuery"
        ]
    ],
    [
        1563066689375,
        "zuora to eloqua.ipynb[30]",
        []
    ],
    [
        1563066689733,
        "zuora to eloqua.ipynb[31]",
        []
    ],
    [
        1563066689737,
        "zuora to eloqua.ipynb[32]",
        []
    ],
    [
        1563066679325,
        "zuora to eloqua.ipynb[4]",
        []
    ],
    [
        1563066679326,
        "zuora to eloqua.ipynb[5]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066679654,
        "zuora to eloqua.ipynb[6]",
        []
    ],
    [
        1563066680626,
        "zuora to eloqua.ipynb[7]",
        [
            "mapDataToFields",
            "mapRatePlanToProduct"
        ]
    ],
    [
        1563066682265,
        "zuora to eloqua.ipynb[8]",
        []
    ]
]

module.exports = exportsCache



#### update export cache?


In [ ]:
var importer = require('../Core')
var updateCode = importer.import('update code cell')

function sortAlphaNumeric(a, b) {
    // convert to strings and force lowercase
    a = typeof a === 'string' ? a.toLowerCase() : a.toString();
    b = typeof b === 'string' ? b.toLowerCase() : b.toString();

    return a.localeCompare(b);
}

function cleanCache(exportsCache) {
    var allCells = importer.interpret()
    var cellIds = allCells.map(cell => cell.id)
    
    // clean up ids that exist in the index but not in memory
    var extra = exportsCache.filter(e => !cellIds.includes(e[1]))
    extra.forEach(e => exportsCache.splice(exportsCache.indexOf(e), 1))
    
    exportsCache.sort((a, b) => {
        return sortAlphaNumeric(a[1], b[1])
    })
}

function updateNotebook(exports) {
    // load current cache
    var exportsCache = importer.import('exports cache')
    var exportIds = exportsCache.map(e => e[1])
    var cacheCell = importer.interpret('exports cache')
    
    // update with results from search
    exports.forEach(e => {
        var i = exportIds.indexOf(e[1])
        if(i === -1) {
            exportsCache.push(e)
            exportIds.push(e[1])
        } else {
            exportsCache.splice(i, 1, e)
        }
    })
    
    cleanCache(exportsCache)
    
    var code = `
// export cache automatically replaced
var exportsCache = ${JSON.stringify(exportsCache, null, 4)}

module.exports = exportsCache
`
    updateCode(cacheCell, code)
}

module.exports = updateNotebook;


#### create export cache?

TODO: generalize this so the database can be anywhere, not just a JSON notebook

TODO: make this a demonstration of idempotence, it keeps loading files.ipynb

TODO: sort and index in an Array instead of object with properties


In [1]:
var path = require('path')
var assert = require('assert')
var importer = require('../Core')
var updateNotebook = importer.import('update export cache')
var getExports = importer.import('get exports from source')

function promiseExports(cell) {
    return new Promise(resolve => {
        setTimeout(() => {
            var result
            try {
                if (process.memoryUsage().heapUsed > 500000000) {
                    throw new Error('out of memory')
                }
                console.log(cell.id)
                result = getExports(cell.code)
            } catch (e) {
                result = e.message
            }
            resolve([Date.now(), cell.id, result])
        }, 100)
    })
}

function createExportCache(search) {
    var exportsCache = importer.import('exports cache')
    var exportIds = exportsCache.map(e => e[1])
    var allCells = importer.interpret()
    var cellIds = allCells.map(cell => cell.id)
    var notebooks = cellIds
        .map(k => k.replace(/\.ipynb\[[0-9]+\]$/ig, '.ipynb'))
        .filter((f, i, arr) => arr.indexOf(f) === i)
    
    // add files that don't exist
    var missing = notebooks.filter(n => !exportIds.includes(`${n}[0]`))
    if(!search) {
        search = missing.pop()
        console.log(`missing ${search}`)
    }
    
    // TODO: update any changed notebooks first
    
    // update the oldest record
    if(!search) {
        exportsCache.sort((a, b) => a[0] - b[0])
        search = exportsCache[0]
        console.log(`updating ${search}`)
    }
    
    if(!search) {
        throw new Error('nothing to do with notebook cache')
    }
    
    var allCells = [].concat.apply([], importer.interpret(search))
    var allCellIds = allCells.map(c => c.id)
    // get only first occurrence
    allCells = allCells
        .filter((c, i) => allCellIds.indexOf(c.id) == i)
        .filter(c => c.code.length < 100000)
    
    console.log(`caching ${search} - ${allCells.length} cells`)

    return Promise
        .all(allCells.map(promiseExports))
        .then(updateNotebook)
}

module.exports = createExportCache

if(typeof $$ !== 'undefined') {
    createExportCache()
}


SyntaxError: Unexpected token ]

#### TODO: cache every notebook in a seperate process?

### notebook cache tools

Some tools for when to update the notebook cache.



#### TODO: stale code

Determine when to update a notebook. Handles the mtime property on cells.

TODO: depends on import cache to build a tree of dependencies


#### update code cell?

Find the code cell following the specified lookup markdown and update the contents.


In [ ]:
var fs = require('fs')

function updateCode(cell, code) {
    var notebook = JSON.parse(fs.readFileSync(cell.filename))
    var match = (/\/\/.*/ig).exec(code)[0]
    
    assert(match && match.length > 2,
           `nothing to match, include a "//" comment`)
    // replace code cell with new code
    // make sure the cell if where the cache says
    assert(notebook.cells[cell.to].source
           .join('')
           .includes(match),
           `code cell "${match}" could not be located`)
    
    notebook.cells[cell.to].source = code.split('\n')
        .map(line => line + '\n')
    
    fs.writeFileSync(cell.filename, JSON.stringify(notebook, null, 4))
}

module.exports = updateCode


## TODO: bookmark cache

Always wanted to write something like evernote to automatically pull in the content of pages I bookmark. Incognito mode on news sites to avoid payment. Ad block plugins installed.  Might as well make ad-blocked cached copies of every page I visit.




## TODO: documentation cache

Tools for searching and downloading documention as it related to this library.

